In [1]:
# 경고창 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install torch transformers

In [3]:
import pymongo
import pandas as pd
from pymongo import MongoClient

# MongoDB 연결하기
client=pymongo.MongoClient("192.168.10.240",27017)
db= client['AI_LKJ']
collection_APP_GOOGLE = db['TRAVEL_APPLICATION']
collection_APP_APPLE = db['TRAVEL_APPLICATION_APPLE']

data_APP_GOOGLE = collection_APP_GOOGLE.find()
data_APP_APPLE = collection_APP_APPLE.find()

df_APP_GOOGLE = pd.DataFrame(data_APP_GOOGLE)
df_APP_APPLE = pd.DataFrame(data_APP_APPLE)

In [4]:
# app(google, apple) 데이터를 concat 진행
df_app = pd.concat([df_APP_GOOGLE, df_APP_APPLE], ignore_index=True)

In [7]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np


load_dataset = df_app["user_comments"]
# 데이터셋이 있다고 가정하고, 데이터셋은 감정 분석을 위해 레이블과 텍스트를 포함해야 합니다.
# 여기서는 Hugging Face의 'datasets' 라이브러리를 사용하여 임시 데이터셋을 생성합니다. 실제 적용 시 데이터셋을 준비해야 합니다.
dataset = load_dataset('user_comments')

# 토크나이저 초기화
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# 모델 초기화
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=6)

# 토큰화 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# 데이터셋 토큰화
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 훈련 및 평가 데이터셋 분할
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(5000))
eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(1000))

# 훈련 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

# 트레이너 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# 트레이너를 사용하여 모델 훈련
trainer.train()

# 감정 분석 예시 함수를 수정한 후의 모델 사용
def evaluate_comment(comment):
    inputs = tokenizer(comment, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=-1)
        return prediction.item()

# 훈련된 모델 사용 예시
comment = "Your text here."
print(evaluate_comment(comment))


ModuleNotFoundError: No module named 'datasets'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 결과 시각화
sns.countplot(x='sentiment', data=df_app)
plt.title('Sentiment Analysis Results')
plt.show()